<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       Deploy Prophet for Time Series Forecast inside Vantage 
<br><br>
   using Script Table Operator
 <br>       
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 150px; height: auto; margin-top: 20pt;">
  <br>
    </p>
</header>

# Use Case: Predict Sales Volume for every product at every store

# Generation of Simulated Data

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random

def generate_sales_data(num_products, num_stores, date_range):
    start_date, end_date = date_range
    date_range = pd.date_range(start=start_date, end=end_date, freq='D')

    # Generate store dimension table
    store_ids = [f"store_{i+1}" for i in range(num_stores)]
    store_names = [f"Store {i+1}" for i in range(num_stores)]
    store_df = pd.DataFrame({
        'store_id': store_ids,
        'store_name': store_names,
        'location': [f"Location {i+1}" for i in range(num_stores)],
        'store_effect': np.random.uniform(0.6, 1.2, num_stores)
    })

    # Generate product dimension table
    product_ids = [f"product_{i+1}" for i in range(num_products)]
    product_names = [f"Product {i+1}" for i in range(num_products)]
    product_df = pd.DataFrame({
        'product_id': product_ids,
        'product_name': product_names,
        'category': [random.choice(['Electronics', 'Clothing', 'Food', 'Toys']) for _ in range(num_products)],
        'product_effect': np.random.uniform(0.1, 12.2, num_products)
    })

    # Weekly and annual seasonality
    weekly_pattern = [0.7, 0.9, 1.0, 1.1, 1.2, 1.3, 1.1]  # Sun to Sat
    annual_pattern = 1.5*np.sin(np.linspace(0, 2 * np.pi, 365)) + 4  # Annual seasonality

    sales_data = []

    # Generate sales data
    for store in store_df.itertuples():
        for product in product_df.itertuples():
            for date in date_range:
                weekly_factor = weekly_pattern[date.weekday()]
                annual_factor = annual_pattern[date.timetuple().tm_yday - 1]
                interaction_effect = (store.store_effect * product.product_effect * weekly_factor * annual_factor)
                base_sales = np.random.poisson(10)
                sales = int(base_sales * interaction_effect*10*np.random.uniform(0.9, 1.1))
                sales = max(sales, 0)  # Ensure no negative sales
                sales_data.append({
                    'store_id': store.store_id,
                    'product_id': product.product_id,
                    'datestamp': date,
                    'items_sold': sales
                })

    sales_df = pd.DataFrame(sales_data)
    
    return sales_df, store_df, product_df

In [ ]:
# Example usage:
sales_df, store_df, product_df = generate_sales_data(10, 5, ('2023-01-01', '2024-01-31'))
# print(sales_df)
# print(store_df)
# print(product_df)

In [ ]:
sales_df.dtypes

In [ ]:
sales_df

# Forecasting with Prophet + Visualisation: Local & Iteratively

In [ ]:
from prophet import Prophet
import pandas as pd

def predict_sales(sales_df, future_days=28):
    predictions = []
    store_product_combinations = sales_df[['store_id', 'product_id']].drop_duplicates()
    
    for _, row in store_product_combinations.iterrows():
        store_id = row['store_id']
        product_id = row['product_id']
        
        # Filter data for the current store and product
        df = sales_df[(sales_df['store_id'] == store_id) & (sales_df['product_id'] == product_id)]
        df = df[['datestamp', 'items_sold']].rename(columns={'datestamp': 'ds', 'items_sold': 'y'})

        # Fit the Prophet model
        model = Prophet(daily_seasonality=True, yearly_seasonality=True)
        model.fit(df)

        # Create future dataframe for predictions
        future = model.make_future_dataframe(periods=future_days)
        
        # Predict future sales
        forecast = model.predict(future)
        
        # Select only the future days and relevant columns
        forecast = forecast[['ds', 'yhat']].tail(future_days)
        forecast['store_id'] = store_id
        forecast['product_id'] = product_id
        
        predictions.append(forecast)

    # Combine all predictions into a single DataFrame
    predictions_df = pd.concat(predictions, ignore_index=True)
    predictions_df.rename(columns={'ds': 'datestamp', 'yhat': 'predicted_items_sold'}, inplace=True)
    
    return predictions_df

# Example usage:
# predictions_df = predict_sales(sales_df)
# print(predictions_df)


In [ ]:
# 50 prophet models fitted and scored iteratively
# will take 50 times model fitting time of a single model
sales_pred_df = predict_sales(sales_df)

In [ ]:
# will have 50* 28 rows
sales_pred_df

# visualise

In [ ]:
import plotly.graph_objects as go

def visualize_sales(sales_df, predicted_sales_df, product_id):
    # Filter the data for the given product
    historic_sales = sales_df[sales_df['product_id'] == product_id]
    predicted_sales = predicted_sales_df[predicted_sales_df['product_id'] == product_id]

    # Get unique stores for the product
    stores = historic_sales['store_id'].unique()

    # Create the plot
    fig = go.Figure()

    # Color palette for the lines
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']

    # Add historic sales lines for each store
    for idx, store in enumerate(stores):
        store_sales = historic_sales[historic_sales['store_id'] == store]
        fig.add_trace(go.Scatter(x=store_sales['datestamp'], y=store_sales['items_sold'],
                                 mode='lines', name=f'{store} Historic',
                                 line=dict(color=colors[idx % len(colors)], width=2)))

    # Add predicted sales lines for each store
    for idx, store in enumerate(stores):
        store_sales = predicted_sales[predicted_sales['store_id'] == store]
        fig.add_trace(go.Scatter(x=store_sales['datestamp'], y=store_sales['predicted_items_sold'],
                                 mode='lines', name=f'{store} Predicted',
                                 line=dict(color=colors[idx % len(colors)], width=2, dash='dot')))

    # Update layout
    fig.update_layout(title=f'Sales Visualization for {product_id}',
                      xaxis_title='Date',
                      yaxis_title='Number of Items Sold',
                      legend_title='Legend')

    fig.show()

# Example usage:
# visualize_sales(sales_df, predictions_df, 'product_1')


In [ ]:
visualize_sales(sales_df, sales_pred_df, 'product_1')

# Run it inside Vantage with Script Table Operator

![](https://nbviewer.org/github/Teradata/jupyter-demos/blob/9fe6c5a18cb85998703538ac3c8dc98e40101f95/UseCases/Store_Sales_Forecasting_Prophet_STO/images/STO.png)

In [ ]:
import getpass
clearscape_host = 'clearscapedemo2024mh-vvthbisb68z9it26.env.clearscape.teradata.com'
clearscape_pw = 'clearscapedemo2024mh'

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings("ignore", category=FutureWarning) 
import pandas as pd
import teradataml as tdml
import json
from IPython.display import display, Markdown

In [ ]:
tdml.__version__

**If you are hosting your Vantage instance on ClearScape Analytics Experience (CSAE) but running the notebook locally, ensure you spin up the environment on CSAE.**

In [ ]:
Param = {'database':'demo_user'}
eng = tdml.create_context(host = clearscape_host,
                          username='demo_user', 
                          password = clearscape_pw)
print(eng)

## (Create) and Upload Demo Data

This step is specific to the demo setup. Under normal circumstances, your raw data would already be stored within Vantage.

In [ ]:
table_name = "sales"

In [ ]:
key = "store_id"

In [ ]:
tdml.copy_to_sql(sales_df, table_name, if_exists="replace", primary_index=key)

In [ ]:
tdml.DataFrame(table_name).tdtypes

# understanding Script Table Operator

## How many AMPs has got our system?

__HASHAMP__

Finds the primary AMP corresponding to the hash bucket number specified in the expression and returns the AMP ID. If no hash bucket number is specified, HASHAMP returns one less than the maximum number of AMPs in the system.

https://docs.teradata.com/r/Enterprise_IntelliFlex_VMware/SQL-Functions-Expressions-and-Predicates/Hash-Related-Functions/HASHAMP

In [ ]:
tdml.DataFrame.from_query("""
SELECT HASHAMP() + 1 AS Number_of_AMPs
""")

## first shell script

In [ ]:
def get_script_query(script_command, distinct= False):
    d = ""
    if distinct:
        d = "DISTINCT"
    return (
f"""
SELECT
 {d} *
FROM
  SCRIPT (
    SCRIPT_COMMAND ('{script_command}') 
    RETURNS ('Message varchar(512)')
  )
""")

In [ ]:
script_command = "echo Hello World!"

In [ ]:
# local execution
!$script_command

In [ ]:
#inside Vantage
tdml.DataFrame.from_query(get_script_query(
    script_command
))

## first python script

In [ ]:
python_code = "print(123)"

In [ ]:
!python -c "$python_code"

__note the python interpreter has the name tdpython3 inside Vantage__

In [ ]:
tdml.DataFrame.from_query(get_script_query(
    f'tdpython3 -c "{python_code}"'
))

## install and run simple python script

In [ ]:
%%writefile print1234.py
print(1234)

In [ ]:
%run print1234.py

In [ ]:
try:
    tdml.execute_sql("call SYSUIF.INSTALL_FILE('print1234', 'print1234.py', 'cz!./print1234.py');")
except:    
    tdml.execute_sql("call SYSUIF.REMOVE_FILE('print1234', 1);")
    tdml.execute_sql("call SYSUIF.INSTALL_FILE('print1234', 'print1234.py', 'cz!./print1234.py');")

In [ ]:
tdml.execute_sql("SET SESSION SEARCHUIFDBPATH = demo_user")

In [ ]:
tdml.DataFrame.from_query(get_script_query(
    f'tdpython3 ./demo_user/print1234.py'
))

# Let's put the prophet script into a format that can be consumed by in database Python

We need to adjust take care of the following parts:

1. reading data from standard input into a pandas dataframe
2. doing the actual payload (modelling and prediction with Prophet)
    - avoid any output to the console from the function
3. putting the result to standard output (print)
4. adjusting the SCRIPT SQL query

### Part 1: Get data in Script
```python
from datetime import datetime
import sys
import pandas as pd
import numpy as np
import time

xcols = ["store_id","product_id","datestamp","items_sold"]

dfIn = pd.read_csv(sys.stdin, sep="\t", header=None, names= xcols,
                   index_col=False, iterator=False, 
                   dtype = {  "items_sold" : np.int64 },
                   parse_dates=["datestamp"]
                  )



# For AMPs that receive no data, exit the script instance gracefully.
if dfIn.empty:
    sys.exit()

    
```

### Part 2: Actuall modelling: Loop within each AMP
```python
from prophet import Prophet

def predict_sales(sales_df, future_days=28):
    predictions = []
    store_product_combinations = sales_df[['store_id', 'product_id']].drop_duplicates()
    
    for _, row in store_product_combinations.iterrows():
        store_id = row['store_id']
        product_id = row['product_id']
        
        # Filter data for the current store and product
        df = sales_df[(sales_df['store_id'] == store_id) & (sales_df['product_id'] == product_id)]
        df = df[['datestamp', 'items_sold']].rename(columns={'datestamp': 'ds', 'items_sold': 'y'})

        # Fit the Prophet model
        model = Prophet(daily_seasonality=True, yearly_seasonality=True)
        model.fit(df)

        # Create future dataframe for predictions
        future = model.make_future_dataframe(periods=future_days)
        
        # Predict future sales
        forecast = model.predict(future)
        
        # Select only the future days and relevant columns
        forecast = forecast[['ds', 'yhat']].tail(future_days)
        forecast['store_id'] = store_id
        forecast['product_id'] = product_id
        
        predictions.append(forecast)

    # Combine all predictions into a single DataFrame
    predictions_df = pd.concat(predictions, ignore_index=True)
    predictions_df.rename(columns={'ds': 'datestamp', 'yhat': 'predicted_items_sold'}, inplace=True)
    
    return predictions_df


sales_pred_df = predict_sales(dfIn.sort_values(['store_id', 'product_id','datestamp']))
    
```

### Part 3: returning everything to standard output via print
```python
for _, row in sales_pred_df.iterrows():
    print('\t'.join(map(str, row.values)))
    
```

## write file & install it

In [ ]:
%%writefile myprophet.py
from datetime import datetime
import sys
import pandas as pd
import numpy as np
import time

xcols = ["store_id","product_id","datestamp","items_sold"]

dfIn = pd.read_csv(sys.stdin, sep="\t", header=None, names= xcols,
                   index_col=False, iterator=False, 
                   dtype = {  "items_sold" : np.int64 },
                   parse_dates=["datestamp"]
                  )



# For AMPs that receive no data, exit the script instance gracefully.
if dfIn.empty:
    sys.exit()

from prophet import Prophet

def predict_sales(sales_df, future_days=28):
    predictions = []
    store_product_combinations = sales_df[['store_id', 'product_id']].drop_duplicates()
    
    for _, row in store_product_combinations.iterrows():
        store_id = row['store_id']
        product_id = row['product_id']
        
        # Filter data for the current store and product
        df = sales_df[(sales_df['store_id'] == store_id) & (sales_df['product_id'] == product_id)]
        df = df[['datestamp', 'items_sold']].rename(columns={'datestamp': 'ds', 'items_sold': 'y'})

        # Fit the Prophet model
        model = Prophet(daily_seasonality=True, yearly_seasonality=True)
        model.fit(df)

        # Create future dataframe for predictions
        future = model.make_future_dataframe(periods=future_days)
        
        # Predict future sales
        forecast = model.predict(future)
        
        # Select only the future days and relevant columns
        forecast = forecast[['ds', 'yhat']].tail(future_days)
        forecast['store_id'] = store_id
        forecast['product_id'] = product_id
        
        predictions.append(forecast)

    # Combine all predictions into a single DataFrame
    predictions_df = pd.concat(predictions, ignore_index=True)
    predictions_df.rename(columns={'ds': 'datestamp', 'yhat': 'predicted_items_sold'}, inplace=True)
    
    return predictions_df


sales_pred_df = predict_sales(dfIn.sort_values(['store_id', 'product_id','datestamp']))

for _, row in sales_pred_df.iterrows():
    print('\t'.join(map(str, row.values)))

In [ ]:
try:
    tdml.execute_sql("call SYSUIF.INSTALL_FILE('myprophet', 'myprophet.py', 'cz!./myprophet.py');")
except:    
    tdml.execute_sql("call SYSUIF.REMOVE_FILE('myprophet', 1);")
    tdml.execute_sql("call SYSUIF.INSTALL_FILE('myprophet', 'myprophet.py', 'cz!./myprophet.py');")

In [ ]:
tdml.execute_sql("SET SESSION SEARCHUIFDBPATH = demo_user")

# Ececute Script

In [ ]:
tdml.DataFrame.from_query("""
SELECT
  TRYCAST(datestamp AS TIMESTAMP) AS datestamp,
  TRYCAST(predicted_items_sold AS FLOAT) AS predicted_items_sold,
  store_id,
  product_id
FROM
  SCRIPT (
    ON  sales
    HASH BY store_id
    SCRIPT_COMMAND ('tdpython3 ./demo_user/myprophet.py') 
    RETURNS ('datestamp VARCHAR(100), predicted_items_sold VARCHAR(100), store_id VARCHAR(100), product_id  VARCHAR(100)')
  )
""").to_sql("sales_forecast", if_exists="replace", primary_index="store_id")

In [ ]:
DF_Forecast = tdml.DataFrame("sales_forecast")
DF_Forecast

In [ ]:
#DF_Forecast.to_pandas().to_csv("forecasts.csv")

# Beyond Prophet

- use STO to separate model fitting and inference, by saving hypersegmented models als BLOB type in a table
- do Image Analysis by reading from Native Object Store
- anything that is possible in Python...

In [ ]:
tdml.remove_context()